# See the diff of (h-bond or non-h-bond) b/w strong and week attention by each head

- hbond combination / Attention strong combination.
- hbond combination / all combination.

In [1]:
%config IPCompleter.use_jedi=False

In [2]:
import os, sys
import subprocess
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.stats import ttest_ind_from_stats
from matplotlib import pyplot as plt

sys.path.append('../streamlit/')
from streamlit_utils import STD_THH, get_attention_and_hhb_relationship
from streamlit_utils import DICT_PDBID_2_CHAINNAMES, get_mapping_2_4digits
from streamlit_utils import DICT_PDBID_2_RESIDUES

from hhbond_stats_byhead import add_4digits

flatten = lambda xlist: [x for xx in xlist for x in xx]
pickleload = lambda p: pickle.load(open(p,"rb"))



loading....


  0%|                                                                                                                                                                                                                   | 0/4 [00:00<?, ?it/s]

ending... loading....


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.70it/s]
27it [00:00, 58.70it/s]

2OL3 has only delta-chain and beta-chain


62it [00:01, 49.62it/s]

4QRR has only delta-chain and beta-chain
5JHD has only delta-chain and beta-chain
5JZI has only delta-chain and beta-chain


79it [00:01, 52.20it/s]
/Users/kyoheikoyama/.pyenv/versions/miniconda3-latest/envs/tcrpred/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [3]:
pd.options.display.float_format = '{:.5g}'.format

In [4]:
%%time

print('STD_THH =', STD_THH)

HEAD_COUNT = 4

# New
distance_path = './residue_distances.parquet'

if os.path.exists(distance_path):
    df_distance = pd.read_parquet(distance_path)
else:
    DATETIME = '20220211_191954'
    DICT_PDBID_2_MELTDIST = pickleload(
        f"/Users/kyoheikoyama/workspace/tcrpred/analysis/DICT_PDB_Result/{DATETIME}_DICT_PDBID_2_MELTDIST_allatoms.pickle"
    )
    df_distance = pd.concat([df.assign(pdbid=k) for k,df in DICT_PDBID_2_MELTDIST.items()])
    df_distance.to_parquet(distance_path)


STD_THH = 4.5
CPU times: user 64.4 ms, sys: 26 ms, total: 90.4 ms
Wall time: 115 ms


In [5]:
NEEDED_PDBIDS = ["2VLK", "5WKF", "3PQY", "4MJI", "4P2Q", "2YPL", "1J8H", "4P2R", "5MEN", "3MV8", "4OZF", "3VXR", "3VXS", "4OZG", "5TEZ", "2J8U", "6Q3S", "4JRX", "3VXU", "1U3H", "4JRY", "4Z7V", "4JFE", "4JFD", "3QIU", "2Z31", "2BNR", "3MBE", "4OZH", "2NX5", "5NHT", "4QOK", "5D2L", "1D9K", "4P2O", "5WKH", "6EQB", "2VLR", "6EQA"]
print(len(NEEDED_PDBIDS))


39


# Only more than x sigma and PosPred Attention 

In [6]:
givenPEP_distributedTCR_by_head_large, givenTCR_distributedPEP_by_head_large =\
get_attention_and_hhb_relationship(strong_atten=True)

df1_strong_atten = pd.concat(flatten([givenPEP_distributedTCR_by_head_large[hi] for hi in range(4)])) 
df2_strong_atten = pd.concat(flatten([givenTCR_distributedPEP_by_head_large[hi] for hi in range(4)]))

givenPEP_distributedTCR_by_head, givenTCR_distributedPEP_by_head =\
get_attention_and_hhb_relationship(strong_atten=False)

df1_normal_atten = pd.concat(flatten([givenPEP_distributedTCR_by_head[hi] for hi in range(4)])) 
df2_normal_atten = pd.concat(flatten([givenTCR_distributedPEP_by_head[hi] for hi in range(4)]))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.53it/s]


# Add minimum distance

In [7]:
dist_tcr_side = pd.merge(df_distance[['pdbid','tcr', 'value']].groupby(['pdbid','tcr'], as_index=False).min(), 
         df_distance, 
         on=['pdbid','tcr', 'value'],
        ).rename(columns={'value':'distance_value', 'peptide':'peptide_min'})

dist_peptide_side = pd.merge(df_distance[['pdbid','peptide', 'value']].groupby(['pdbid','peptide'], as_index=False).min(), 
         df_distance, 
         on=['pdbid','peptide', 'value'],
        ).rename(columns={'value':'distance_value', 'tcr':'tcr_min'})


df1_strong_atten = pd.merge(df1_strong_atten, dist_tcr_side, on=['pdbid','tcr'], how='left')
df1_normal_atten = pd.merge(df1_normal_atten, dist_tcr_side, on=['pdbid','tcr'], how='left' )
df2_strong_atten = pd.merge(df2_strong_atten, dist_peptide_side, on=['pdbid','peptide'], how='left' )
df2_normal_atten = pd.merge(df2_normal_atten, dist_peptide_side, on=['pdbid','peptide'], how='left' )

df1_strong_atten = df1_strong_atten.query('pdbid in @NEEDED_PDBIDS')
df1_normal_atten = df1_normal_atten.query('pdbid in @NEEDED_PDBIDS')
df2_strong_atten = df2_strong_atten.query('pdbid in @NEEDED_PDBIDS')
df2_normal_atten = df2_normal_atten.query('pdbid in @NEEDED_PDBIDS')

df_tcr_side = pd.concat([df1_strong_atten.assign(is_large_atten=True), df1_normal_atten.assign(is_large_atten=False), ])
df_tcr_side = df_tcr_side.assign(is_tcr_side=True)
df_pep_side = pd.concat([df2_strong_atten.assign(is_large_atten=True), df2_normal_atten.assign(is_large_atten=False), ])
df_pep_side = df_pep_side.assign(is_tcr_side=False)
df_pep_side

,peptide,tcr,value_x,value_y,attention_original,attention_log10,pdbid,head,distance_value,tcr_min,is_large_atten,is_tcr_side
0,I_7,A_14,13.643,0.54368,0.54368,-0.26466,2YPL,0,3.2118,Y_6,True,False
1,V_3,S_17,21.323,0.44758,0.44758,-0.34913,4P2R,0,3.2836,N_6,True,False
2,L_5,A_21,3.5747,0.5087,0.5087,-0.29354,4OZF,0,3.5747,A_21,True,False
3,R_8,C_14,20.871,0.68302,0.68302,-0.16557,1U3H,0,2.61,Y_23,True,False
4,G_7,Y_23,10.17,0.48898,0.48898,-0.31071,4JRY,0,2.9548,R_18,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
55257,V_8,T_21,13.218,0.096361,0.096361,-1.0161,6EQA,3,7.7018,T_17,False,False
55258,V_8,T_9,24.827,0.025839,0.025839,-1.5877,6EQA,3,7.7018,T_17,False,False
55259,V_8,V_2,23.696,0.040502,0.040502,-1.3925,6EQA,3,7.7018,T_17,False,False
55260,V_8,V_4,21.409,0.023304,0.023304,-1.6326,6EQA,3,7.7018,T_17,False,False


# Compare Properties

## Take the min by each head

In [8]:
DICT_bond_tuple_fromto = {}

from streamlit_utils import read_hhb_file, get_mapping_2_4digits, DICT_PDBID_2_CHAINNAMES

def read_hhb_text_and_find_chain(text, chainsletters):
    ress_atoms = text[:45].replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')
    donor_res = ress_atoms[0]
    acceptor_res = ress_atoms[2]
    if donor_res.split('/')[1][:1] in chainsletters or \
            acceptor_res.split('/')[1][:1] in chainsletters:
        return text


def create_bond_info_by_pdb(pdbid):
    chains = DICT_PDBID_2_CHAINNAMES[pdbid.upper()]
    chainsletters = ''.join(chains)

    dfhhb = read_hhb_file(pdbid)
    donors_n_acceptors = [read_hhb_text_and_find_chain(row.item(), chainsletters) for i, row in dfhhb.iterrows()]
    donors_n_acceptors = [a for a in donors_n_acceptors if a is not None]
    
    bond_tuple_fromto = []
    for da in donors_n_acceptors:
        a = da.replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')
        from_d = a[0].split('/')[1].split('-')[0][:5]
        to_d = a[2].split('/')[1].split('-')[0][:5]
        bond_tuple_fromto.append((from_d, to_d))
        
    DICT_bond_tuple_fromto[pdbid] = bond_tuple_fromto
    anyhbond_related_4digits = list(set([tup[0] for tup in bond_tuple_fromto] + [tup[1] for tup in bond_tuple_fromto]))

    CDRname_2_4digits, Peptide_2_4digits, bond_list = get_mapping_2_4digits(donors_n_acceptors, pdbid)
    dfc = pd.DataFrame(CDRname_2_4digits.items(), columns=['residue', 'digit4']).assign(**{'is_tcr':True, 'pdbid':pdbid})
    dfp = pd.DataFrame(Peptide_2_4digits.items(), columns=['residue', 'digit4']).assign(**{'is_tcr':False, 'pdbid':pdbid})
    df = pd.concat([dfc,dfp])
    df['has_bond'] = df['digit4'].isin(anyhbond_related_4digits)
    return df
    

In [9]:

def generate_alpha_beta_pep():
    alis, blis, clis = [],[],[]
    for pdbid in NEEDED_PDBIDS:
        residues = DICT_PDBID_2_RESIDUES[pdbid]
        df_alpha = pd.DataFrame([residues[0]]).T.assign(pdbid=pdbid)
        df_alpha['digit4'] = df_alpha[0].apply(add_4digits)
        df_beta = pd.DataFrame([residues[1]]).T.assign(pdbid=pdbid)
        df_beta['digit4'] = df_beta[0].apply(add_4digits)
        df_pep = pd.DataFrame([residues[2]]).T.assign(pdbid=pdbid)
        df_pep['digit4'] = df_pep[0].apply(add_4digits)
        alis.append(df_alpha)
        blis.append(df_beta)
        clis.append(df_pep)
    return pd.concat(alis),pd.concat(blis),pd.concat(clis)

df_alpha, df_beta, df_pep = generate_alpha_beta_pep()

In [10]:
df_bondinfo = pd.concat([create_bond_info_by_pdb(p) for p in NEEDED_PDBIDS])
df_bond_tuple_fromto = pd.DataFrame(DICT_bond_tuple_fromto.items(), columns=['pdbid', 'bonds'])

df_bond_relations = []
for i, row in df_bond_tuple_fromto.iterrows():
    list_of_tuple = row['bonds']
    pdbid = row['pdbid']
    bond_dict = defaultdict(list)
    for tup in list_of_tuple:
        bond_dict[tup[0]].append(tup[1])
        bond_dict[tup[1]].append(tup[0])
    t = pd.DataFrame(bond_dict.items(), columns=['from_res', 'to_res_list']).assign(pdbid=pdbid)
    df_bond_relations.append(t)
df_bond_relations = pd.concat(df_bond_relations)


In [11]:
df_bondinfo[df_bondinfo.is_tcr==False]

,residue,digit4,is_tcr,pdbid,has_bond
0,G_0,C0001,False,2VLK,True
1,I_1,C0002,False,2VLK,True
2,L_2,C0003,False,2VLK,True
3,G_3,C0004,False,2VLK,True
4,F_4,C0005,False,2VLK,True
...,...,...,...,...,...
4,G_4,C0006,False,6EQA,True
5,I_5,C0007,False,6EQA,True
6,L_6,C0008,False,6EQA,False
7,T_7,C0009,False,6EQA,True


# Create info

In [12]:
df_pep_side.head(3) # info of tcr-side attention given a pdbid and given a head

,peptide,tcr,value_x,value_y,attention_original,attention_log10,pdbid,head,distance_value,tcr_min,is_large_atten,is_tcr_side
0,I_7,A_14,13.643,0.54368,0.54368,-0.26466,2YPL,0,3.2118,Y_6,True,False
1,V_3,S_17,21.323,0.44758,0.44758,-0.34913,4P2R,0,3.2836,N_6,True,False
2,L_5,A_21,3.5747,0.5087,0.5087,-0.29354,4OZF,0,3.5747,A_21,True,False


In [13]:
df_bondinfo = pd.merge(df_bondinfo, df_bond_relations, 
                       left_on=['pdbid', 'digit4'], right_on=['pdbid', 'from_res'], 
                       how='left')

df_bondinfo['to_res_list'] = df_bondinfo.to_res_list.apply(lambda xlis: [] if not isinstance(xlis, list) else xlis)

In [14]:
# is_connecting_tcr
df_bondinfo['is_connecting_to_cdr'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if isinstance(row.to_res_list, list):
        p = row.pdbid
        cdr_list = df_bondinfo.query('is_tcr==True and pdbid==@p')['digit4'].tolist()
        is_any_res_in_cdr = any([(r in cdr_list) for r in row.to_res_list])
        df_bondinfo.loc[[i], 'is_connecting_to_cdr'] = is_any_res_in_cdr
    
del cdr_list, is_any_res_in_cdr

1474it [00:03, 428.81it/s]


In [15]:
df_bondinfo['is_connecting_to_pep'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if isinstance(row.to_res_list, list):
        p = row.pdbid
        pep_list = df_bondinfo.query('is_tcr==False and pdbid==@p')['digit4'].tolist()
        is_any_res_in_pep = any([(r in pep_list) for r in row.to_res_list])
        df_bondinfo.loc[[i], 'is_connecting_to_pep']  = is_any_res_in_pep
del pep_list, is_any_res_in_pep

1474it [00:03, 448.87it/s]


In [16]:
df_bondinfo['is_connecting_to_tcr'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if isinstance(row.to_res_list, list):
        p = row.pdbid
        tcr_list =\
            [t for t in df_alpha.query('pdbid==@p')['digit4'].tolist()] +\
            [t for t in df_beta.query('pdbid==@p')['digit4'].tolist()]
        is_tcr_list = any([(r in tcr_list) for r in row.to_res_list])
        df_bondinfo.loc[[i], 'is_connecting_to_tcr']  = is_tcr_list
        
del tcr_list, is_tcr_list

1474it [00:04, 321.91it/s]


In [17]:
df_bondinfo['is_connecting_to_notCDR_tcr'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if isinstance(row.to_res_list, list):
        p = row.pdbid
        cdr_list = df_bondinfo.query('is_tcr==True and pdbid==@p')['digit4'].tolist()
        notcdr_tcr_list =\
            [t for t in df_alpha.query('pdbid==@p')['digit4'].tolist() if t not in cdr_list] +\
            [t for t in df_beta.query('pdbid==@p')['digit4'].tolist() if t not in cdr_list]
        is_notcdr_tcr_list = any([(r in notcdr_tcr_list) for r in row.to_res_list])
        df_bondinfo.loc[[i], 'is_connecting_to_notCDR_tcr']  = is_notcdr_tcr_list
del notcdr_tcr_list, is_notcdr_tcr_list


1474it [00:07, 201.76it/s]


In [18]:
df_bondinfo['num_bonds'] = df_bondinfo['to_res_list'].apply(len)

In [19]:
df_bondinfo['digit4_is_in_edge'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    p = row.pdbid
    EDGENUM = 3
    residue_list = df_bondinfo.query('is_tcr==False and pdbid==@p')['residue'].tolist()
    beginnings_endings_cdrs = residue_list[:EDGENUM] + residue_list[-EDGENUM:]
    df_bondinfo.loc[[i], 'digit4_is_in_edge']  = row.residue in beginnings_endings_cdrs


1474it [00:03, 386.78it/s]


In [20]:
df_bondinfo['is_connecting_to_cdr'].value_counts(),\
df_bondinfo['is_connecting_to_pep'].value_counts(),\
df_bondinfo['is_connecting_to_notCDR_tcr'].value_counts()

(False    957
 True     517
 Name: is_connecting_to_cdr, dtype: int64,
 False    1362
 True      112
 Name: is_connecting_to_pep, dtype: int64,
 False    972
 True     502
 Name: is_connecting_to_notCDR_tcr, dtype: int64)

# Merge

In [21]:
df_pep_byhead = pd.concat([
    df_pep_side.groupby(['pdbid', 'peptide', 'head']).max()[['is_large_atten',]],
    df_pep_side.groupby(['pdbid', 'peptide', 'head']).min()[['distance_value',]],
                          ], axis=1).reset_index()

df_pep_byhead = pd.merge(df_pep_byhead,
                         df_bondinfo.query('is_tcr!=True'), 
                         how='left', 
                         left_on=['pdbid', 'peptide'], 
                         right_on=['pdbid', 'residue'], )

df_pep_allhead = pd.concat([
    df_pep_side.groupby(['pdbid', 'peptide', ]).max()[['is_large_atten',]],
    df_pep_side.groupby(['pdbid', 'peptide', ]).min()[['distance_value',]],
                          ], axis=1).reset_index()

df_pep_allhead = pd.merge(df_pep_allhead, df_bondinfo.query('is_tcr!=True'), how='left', 
             left_on=['pdbid', 'peptide'], 
             right_on=['pdbid', 'residue'], )

# Contingency table and Fisher's test

In [22]:
from scipy import stats


In [23]:
prop_list = ['is_connecting_to_cdr', 'is_connecting_to_pep', 
             'is_connecting_to_tcr', 'is_connecting_to_notCDR_tcr',
             'digit4_is_in_edge', 'distance_value', 'num_bonds'
            ]

In [24]:
print([c for c in df_pep_allhead.columns if 'is_' in c])

['is_large_atten', 'is_tcr', 'is_connecting_to_cdr', 'is_connecting_to_pep', 'is_connecting_to_tcr', 'is_connecting_to_notCDR_tcr', 'digit4_is_in_edge']


## All heads combined

In [25]:
for prop in prop_list:
    print()
    print(prop)
    pd.options.display.float_format = '{:.3g}'.format
    means = (df_pep_allhead.groupby('is_large_atten').mean().sort_index(ascending=False)[prop]).values
    stds = (df_pep_allhead.groupby('is_large_atten').std().sort_index(ascending=False)[prop]).values
    obs = df_pep_allhead.groupby('is_large_atten').agg(len).sort_index(ascending=False).iloc[:,0].values
    ttest_result = stats.ttest_ind_from_stats(mean1=means[0], mean2=means[1], 
                               std1=stds[0], std2=stds[1],
                               nobs1=obs[0], nobs2=obs[1],
                              )
    print('  ', ttest_result)
    print(f"\t mean+std of large = {means[0]}+-{stds[0]}", f'n_obs =',obs[0])
    print(f"\t mean+std of small = {means[1]}+-{stds[1]}", f'n_obs =',obs[1] )
    



is_connecting_to_cdr
   Ttest_indResult(statistic=1.2037150191395052, pvalue=0.2293543677036356)
	 mean+std of large = 0.2222222222222222+-0.41833001326703784 n_obs = 81
	 mean+std of small = 0.16573033707865167+-0.37236174336542627 n_obs = 356

is_connecting_to_pep
   Ttest_indResult(statistic=-0.5126240398392281, pvalue=0.608474529038187)
	 mean+std of large = 0.04938271604938271+-0.21801574300387316 n_obs = 81
	 mean+std of small = 0.06460674157303371+-0.24617665101247713 n_obs = 356

is_connecting_to_tcr
   Ttest_indResult(statistic=1.937534004551377, pvalue=0.053327028269777095)
	 mean+std of large = 0.3333333333333333+-0.4743416490252569 n_obs = 81
	 mean+std of small = 0.2303370786516854+-0.42164119166493286 n_obs = 356

is_connecting_to_notCDR_tcr
   Ttest_indResult(statistic=0.6687374462980734, pvalue=0.5040176948412514)
	 mean+std of large = 0.13580246913580246+-0.34471315683307935 n_obs = 81
	 mean+std of small = 0.10955056179775281+-0.3127683212673814 n_obs = 356

digit4_i

## All heads combined

In [26]:
for prop in prop_list:
    if prop=='distance_value' or prop=='num_bonds':
        continue
    print()
    print('property', prop)
    df_margin = pd.crosstab(df_pep_allhead.is_large_atten, df_pep_allhead[prop], margins=True)
    display(df_margin)
    df = pd.crosstab(df_pep_allhead.is_large_atten, df_pep_allhead[prop], dropna=False)
    oddsr, p = stats.fisher_exact(table=df.to_numpy(), alternative='two-sided')
    
    print(f'oddsr={oddsr:.5f}, p={p:.5f}')
    if p<0.1:
        print(f'Significant! head=all, prop={prop}, p={p}')
    


property is_connecting_to_cdr


is_connecting_to_cdr,False,True,All
is_large_atten,,,
False,297,59,356
True,63,18,81
All,360,77,437


oddsr=1.43826, p=0.25769

property is_connecting_to_pep


is_connecting_to_pep,False,True,All
is_large_atten,,,
False,333,23,356
True,77,4,81
All,410,27,437


oddsr=0.75212, p=0.79947

property is_connecting_to_tcr


...

oddsr=1.67073, p=0.06406
Significant! head=all, prop=is_connecting_to_tcr, p=0.06405699764774328

property is_connecting_to_notCDR_tcr


is_connecting_to_notCDR_tcr,False,True,All
is_large_atten,,,
False,317,39,356
True,70,11,81
All,387,50,437


oddsr=1.27729, p=0.56122

property digit4_is_in_edge


digit4_is_in_edge,False,True,All
is_large_atten,,,
False,154,202,356
True,49,32,81
All,203,234,437


oddsr=0.49788, p=0.00642
Significant! head=all, prop=digit4_is_in_edge, p=0.006423595453990703


In [27]:
print('mean', df_pep_allhead.groupby('is_large_atten').mean()[['distance_value']])
print('std', df_pep_allhead.groupby('is_large_atten').std()[['distance_value']])
print()
print(stats.ttest_ind(
    df_pep_allhead.query('is_large_atten==True')['distance_value'],
    df_pep_allhead.query('is_large_atten==False')['distance_value']
))

mean                 distance_value
is_large_atten                
False                     5.36
True                      4.52
std                 distance_value
is_large_atten                
False                     2.91
True                      2.16

Ttest_indResult(statistic=-2.4460679731342276, pvalue=0.01483755573700187)


## By head

In [28]:
from scipy import stats
for hhh in range(4):
    print('-'*30)
    print('head', hhh)
    for prop in prop_list + ['distance_value']:
        temp = df_pep_byhead.query('head==@hhh')
        if prop=='distance_value' or prop=='num_bonds':
            pd.options.display.float_format = '{:.3g}'.format
            print('prop', prop)
            display(temp.groupby('is_large_atten').mean().sort_index(ascending=False)[['distance_value']].T)
            continue
        print()
        print('property', prop)
        df_margin = pd.crosstab(temp.is_large_atten, temp[prop], margins=True)
        display(df_margin)
        df = pd.crosstab(temp.is_large_atten, temp[prop], margins=False)
        if True not in df.columns:df[True]=[0,0]
        oddsr, p = stats.fisher_exact(table=df.to_numpy(), alternative='two-sided')
        print(f'oddsr={oddsr:.5f}, p={p:.5f}')
        
        
        if p<0.1:
            print(f'Significant! head={hhh}, prop={prop}, p={p}')


------------------------------
head 0

property is_connecting_to_cdr


is_connecting_to_cdr,False,True,All
is_large_atten,,,
False,348,76,424
True,12,1,13
All,360,77,437


oddsr=0.38158, p=0.48055

property is_connecting_to_pep


is_connecting_to_pep,False,True,All
is_large_atten,,,
False,398,26,424
True,12,1,13
All,410,27,437


oddsr=1.27564, p=0.56876

property is_connecting_to_tcr


...

is_large_atten,True,False
distance_value,3.74,5.29


prop num_bonds


is_large_atten,True,False
distance_value,3.74,5.29


prop distance_value


is_large_atten,True,False
distance_value,3.74,5.29


# Student t-test by PDB

## all-head

In [29]:
def return_property_by_islarge(temp, prop):
    if prop not in ['distance_value', 'num_bonds']:
        temp = temp.groupby(['pdbid', 'peptide', ]).max().reset_index()
        df = pd.crosstab(temp.is_large_atten, temp[prop], margins=False, )
        if True not in df.columns:
            df[True] = [0 for _ in range(len(df))]
        if (len(df.columns)!=2) or (len(df.index)!=2):
            return [np.nan, (df[True] / df.sum(axis=1)).loc[False]]
            
        df_margin = pd.crosstab(temp.is_large_atten, temp[prop], margins=True, normalize=True)
        oddsr, p = stats.fisher_exact(table=df.to_numpy(), alternative='two-sided')
        return (df[True] / df.sum(axis=1)).sort_index(ascending=False).values
    else:
        temp = pd.concat([
            temp.groupby(['peptide', ]).max()[['is_large_atten','digit4_is_in_edge']],
            temp.groupby(['peptide', ]).min()[['distance_value',]],
            temp.groupby(['peptide', ]).mean()[['num_bonds',]],
        ], axis=1).reset_index()

        temp_mean = temp.groupby('is_large_atten').mean()[prop]
        if len(temp_mean)==1:
            return  [np.nan, temp_mean.loc[False]]
        return [temp_mean.loc[True], temp_mean.loc[False]]
    

In [30]:
prop_list = ['is_connecting_to_pep', 'is_connecting_to_cdr', 
             #'is_connecting_to_ownchain_tcr','is_connecting_to_ownchain_cdr',
             # 'is_connecting_to_opposite_chain_tcr', 'is_connecting_to_opposite_chain_cdr',
             'is_connecting_to_tcr', 'is_connecting_to_notCDR_tcr',
             'digit4_is_in_edge', 'distance_value', 'num_bonds',
            ]

for prop in prop_list:
    vals = []
    vals2 = []
    print('-'*30)
    for ppp in df_pep_byhead.pdbid.unique():     
        temp = df_pep_byhead.query('pdbid==@ppp')
        vals2.append(return_property_by_islarge(temp, prop))

    vals = np.array(vals2)
    ttest_result = stats.ttest_rel(vals[:,0], vals[:,1],nan_policy='omit')
    print('prop', prop)
    #     means = np.array(vals).mean(axis=0).tolist()
    #     stds = np.array(vals).std(axis=0)
    means = np.nanmean(vals, axis=0).tolist()
    stds = np.nanstd(vals, axis=0)
    
    if np.isnan(means[0]):
        assert False
    print(f"mean+std of large = {means[0]}+-{stds[0]}", )
    print(f"mean+std of small = {means[1]}+-{stds[1]}", )
    print("std of (large, small) = ", )
    print('ttest_result', ttest_result)
    if ttest_result[1]<0.05:
        print('*** Significant!')
    else:
        print('#### not significant...')

------------------------------
prop is_connecting_to_pep
mean+std of large = 0.04954954954954955+-0.1442848412436664
mean+std of small = 0.06588504088504088+-0.12055723048629587
std of (large, small) = 
ttest_result Ttest_relResult(statistic=-0.7501749310873257, pvalue=0.45802445307036)
#### not significant...
------------------------------
prop is_connecting_to_cdr
mean+std of large = 0.20495495495495497+-0.30238950442328233
mean+std of small = 0.1681590845052383+-0.09818237744365198
std of (large, small) = 
ttest_result Ttest_relResult(statistic=0.7138043834711818, pvalue=0.4799506747999377)
#### not significant...
------------------------------
prop is_connecting_to_tcr
mean+std of large = 0.3400900900900901+-0.3713553509626332
mean+std of small = 0.23718219102834487+-0.11842391588627253
std of (large, small) = 
ttest_result Ttest_relResult(statistic=1.4677907587095518, pvalue=0.15084445427698812)
#### not significant...
------------------------------
prop is_connecting_to_notCDR_tc

In [31]:
table_all = []
for prop in prop_list:
    vals = []
    vals2 = []
    print('-'*30)
    for ppp in df_pep_byhead.pdbid.unique():     
        temp = df_pep_byhead.query('pdbid==@ppp')
        vals2.append(return_property_by_islarge(temp, prop))

    vals = np.array(vals2)
    ttest_result = stats.ttest_rel(vals[:,0], vals[:,1],nan_policy='omit')
    print('prop', prop)
    means = np.nanmean(vals, axis=0).tolist()
    stds = np.nanstd(vals, axis=0)
    if np.isnan(means[0]):
        assert False
    table_all.append(['Proportion ' + prop if 'is_' in prop else prop, 
                 f"{means[0]:.4f}+-{stds[0]:.4f}", 
                 f"{means[1]:.4f}+-{stds[1]:.4f}", 
                 ttest_result[1],
                 'all']
                )        
    print(f"mean+std of large = {means[0]}+-{stds[0]}", )
    print(f"mean+std of small = {means[1]}+-{stds[1]}", )
    print("std of (large, small) = ", )
    print('ttest_result', ttest_result)
    if ttest_result[1]<0.05:
        print('*** Significant!')
    else:
        print('#### not significant...')

------------------------------
prop is_connecting_to_pep
mean+std of large = 0.04954954954954955+-0.1442848412436664
mean+std of small = 0.06588504088504088+-0.12055723048629587
std of (large, small) = 
ttest_result Ttest_relResult(statistic=-0.7501749310873257, pvalue=0.45802445307036)
#### not significant...
------------------------------
prop is_connecting_to_cdr
mean+std of large = 0.20495495495495497+-0.30238950442328233
mean+std of small = 0.1681590845052383+-0.09818237744365198
std of (large, small) = 
ttest_result Ttest_relResult(statistic=0.7138043834711818, pvalue=0.4799506747999377)
#### not significant...
------------------------------
prop is_connecting_to_tcr
mean+std of large = 0.3400900900900901+-0.3713553509626332
mean+std of small = 0.23718219102834487+-0.11842391588627253
std of (large, small) = 
ttest_result Ttest_relResult(statistic=1.4677907587095518, pvalue=0.15084445427698812)
#### not significant...
------------------------------
prop is_connecting_to_notCDR_tc

In [32]:
table_all = pd.DataFrame(table_all, 
             columns=['property', 'Large Atten Mean. Mean(STD)', 'Small Atten. Mean(STD)', 'p value', 'Head'])
table_all.loc[table_all['p value'] < 0.10, ' '] = '*'
table_all.loc[table_all['p value'] < 0.05, ' '] = '***'
table_all.loc[table_all['p value'] >= 0.05, ' '] = ''
table_all[' '].fillna(' ')
table_all

,property,Large Atten Mean. Mean(STD),Small Atten. Mean(STD),p value,Head,
0,Proportion is_connecting_to_pep,0.0495+-0.1443,0.0659+-0.1206,0.458,all,
1,Proportion is_connecting_to_cdr,0.2050+-0.3024,0.1682+-0.0982,0.48,all,
2,Proportion is_connecting_to_tcr,0.3401+-0.3714,0.2372+-0.1184,0.151,all,
3,Proportion is_connecting_to_notCDR_tcr,0.1712+-0.3112,0.1118+-0.1283,0.355,all,
4,Proportion digit4_is_in_edge,0.4459+-0.4097,0.5874+-0.1232,0.0795,all,
5,distance_value,4.6398+-1.7149,5.1926+-1.2647,0.141,all,
6,num_bonds,2.1126+-1.4959,2.0031+-0.9051,0.668,all,


## By head

In [33]:
prop_list = ['distance_value', 'num_bonds', 'is_connecting_to_cdr', 'is_connecting_to_pep', 
             'is_connecting_to_tcr', 'is_connecting_to_notCDR_tcr',
             'digit4_is_in_edge', 
            ]


In [34]:
table = []
table_by_pdb = []
for hhh in range(4):
    print()
    print()
    print('head',hhh, '*'*30)
    df_by_prop = []
    for prop in prop_list:
        vals = []
        for ppp in df_pep_byhead.pdbid.unique():     
            temp = df_pep_byhead.query('pdbid==@ppp and head==@hhh')
            vals.append(return_property_by_islarge(temp, prop))
        
        vals = np.array(vals)
        
        df_minimum = pd.DataFrame(vals, index=df_pep_byhead.pdbid.unique(), columns=[f'Large {prop}', f'Small {prop}'])
        
        ttest_result = stats.ttest_rel(vals[:,0], vals[:,1], nan_policy='omit')
        means = np.nanmean(vals, axis=0).tolist()
        stds = np.nanstd(vals, axis=0)
        if np.isnan(means[0]):
            assert False
        print()
        print('  prop', prop)
        print(f"\t mean+std of large = {means[0]}+-{stds[0]}", )
        print(f"\t mean+std of small = {means[1]}+-{stds[1]}", )
        # print("std of (large, small) = ", )
        print(ttest_result)
        if ttest_result[1]<0.05:
            print(f'*** Significant! head={hhh}, p={ttest_result[1]}, prop={prop}')
        else:
            print('- not significant...')
                
        table.append(['Proportion ' + prop if 'is_' in prop else prop, 
                     f"{means[0]:.4f}+-{stds[0]:.4f}", 
                     f"{means[1]:.4f}+-{stds[1]:.4f}", 
                     ttest_result[1],
                     hhh]
                    )
        df_by_prop.append(df_minimum)
    table_by_pdb.append(pd.concat(df_by_prop,axis=1).assign(head=hhh))
    



head 0 ******************************

  prop distance_value
	 mean+std of large = 4.123303284247716+-1.3970292097187857
	 mean+std of small = 5.079643427432243+-1.1194058915384217
Ttest_relResult(statistic=-2.003764781601175, pvalue=0.07034862765722524)
- not significant...

  prop num_bonds
	 mean+std of large = 1.6666666666666667+-1.5986105077709065
	 mean+std of small = 2.012536319889261+-0.7578816526002904
Ttest_relResult(statistic=-0.20618788209670616, pvalue=0.8404107713697226)
- not significant...

  prop is_connecting_to_cdr
	 mean+std of large = 0.08333333333333333+-0.27638539919628324
	 mean+std of small = 0.18006266818484012+-0.10857580820136695
Ttest_relResult(statistic=-0.8584488431429932, pvalue=0.4089610273050542)
- not significant...

  prop is_connecting_to_pep
	 mean+std of large = 0.08333333333333333+-0.27638539919628324
	 mean+std of small = 0.0656386348694041+-0.12181223077104664
Ttest_relResult(statistic=0.0, pvalue=1.0)
- not significant...

  prop is_connecti

In [35]:
df = pd.DataFrame(table, 
             columns=['property', 'Large Atten Mean. Mean(STD)', 'Small Atten. Mean(STD)', 'p value', 'Head'])
df.loc[df['p value'] < 0.10, ' '] = '*'
df.loc[df['p value'] < 0.05, ' '] = '***'
df.loc[df['p value'] >= 0.05, ' '] = ''
df[' '].fillna(' ')
df

,property,Large Atten Mean. Mean(STD),Small Atten. Mean(STD),p value,Head,
0,distance_value,4.1233+-1.3970,5.0796+-1.1194,0.0703,0,
1,num_bonds,1.6667+-1.5986,2.0125+-0.7579,0.84,0,
2,Proportion is_connecting_to_cdr,0.0833+-0.2764,0.1801+-0.1086,0.409,0,
3,Proportion is_connecting_to_pep,0.0833+-0.2764,0.0656+-0.1218,1,0,
4,Proportion is_connecting_to_tcr,0.3333+-0.4714,0.2559+-0.1198,0.552,0,
5,Proportion is_connecting_to_notCDR_tcr,0.2500+-0.4330,0.1167+-0.1116,0.318,0,
6,Proportion digit4_is_in_edge,0.2500+-0.4330,0.5637+-0.0951,0.0504,0,
7,distance_value,5.1794+-2.4525,5.0293+-1.0674,0.895,1,
8,num_bonds,2.4348+-1.7087,1.9672+-0.7934,0.154,1,
9,Proportion is_connecting_to_cdr,0.3261+-0.4570,0.1710+-0.0996,0.215,1,


In [37]:
pd.concat([df, table_all])

,property,Large Atten Mean. Mean(STD),Small Atten. Mean(STD),p value,Head,
0,distance_value,4.1233+-1.3970,5.0796+-1.1194,0.0703,0,
1,num_bonds,1.6667+-1.5986,2.0125+-0.7579,0.84,0,
2,Proportion is_connecting_to_cdr,0.0833+-0.2764,0.1801+-0.1086,0.409,0,
3,Proportion is_connecting_to_pep,0.0833+-0.2764,0.0656+-0.1218,1,0,
4,Proportion is_connecting_to_tcr,0.3333+-0.4714,0.2559+-0.1198,0.552,0,
5,Proportion is_connecting_to_notCDR_tcr,0.2500+-0.4330,0.1167+-0.1116,0.318,0,
6,Proportion digit4_is_in_edge,0.2500+-0.4330,0.5637+-0.0951,0.0504,0,
7,distance_value,5.1794+-2.4525,5.0293+-1.0674,0.895,1,
8,num_bonds,2.4348+-1.7087,1.9672+-0.7934,0.154,1,
9,Proportion is_connecting_to_cdr,0.3261+-0.4570,0.1710+-0.0996,0.215,1,
